In [31]:
import torch
from torchvision.datasets import CIFAR10
import sys
sys.path.append('..')
from registry import MODEL_DICT, NORMALIZE_DICT, RatioCIFAR10
torch.backends.cudnn.deterministic = True
torch.cuda.manual_seed(0)
torch.manual_seed(0)

In [32]:
import torchvision.transforms as T
name = 'cifar10'
data_root = '/data/lijingru/cifar10/'
train_transform = T.Compose([
    #T.Resize((224, 224), Image.BICUBIC),
    T.RandomCrop(32, padding=4),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize( **NORMALIZE_DICT[name] ),
])
val_transform = T.Compose([
    #T.Resize((224, 224), Image.BICUBIC),
    T.ToTensor(),
    T.Normalize( **NORMALIZE_DICT[name] ),
])
train_dst = CIFAR10(data_root, train=True, download=True, transform=val_transform)


Files already downloaded and verified


In [33]:
import registry
device = 'cuda:1'
model = registry.get_model(name='resnet34', num_classes=10)
model.load_state_dict(torch.load('/data/lijingru/DataFree/checkpoints/scratch/cifar10_resnet34.pth', map_location='cpu')['state_dict'])
model = model.to(device)

In [4]:
ratio_train_dst = RatioCIFAR10(model='resnet34', dst=train_dst, ratio=1.0, gpu=device)

Updating Confidence indices...: 100%|██████████| 391/391 [00:09<00:00, 41.69it/s]


Sample 50000 samples


In [34]:
dataloader1 = torch.utils.data.DataLoader(train_dst, batch_size=128, shuffle=False, num_workers=8)
# dataloader2 = torch.utils.data.DataLoader(ratio_train_dst, batch_size=128, shuffle=False, num_workers=8, pin_memory=True)

In [18]:
x, y, indice = ratio_train_dst._confidence()

NameError: name 'ratio_train_dst' is not defined

In [7]:
ratio_train_dst.indice

tensor([36921, 36083, 16663,  ..., 15849, 45184,  9642])

In [37]:
import tqdm
res = {
    'x': [],
    'y': [],
    'ent': []
}
# dataloader = torch.utils.data.DataLoader(self.dst, batch_size=128, shuffle=False, num_workers=8, pin_memory=True)
for x,y in tqdm.tqdm(dataloader1, desc='Updating Confidence indices...'):
    output_logit = model(x.to(device))
    p = (output_logit).softmax(1)
    ents = (p * p.log()).sum(1)
    res['x'] += list(x)
    res['y'] += list(y)
    res['ent'] += list(ents.detach().cpu())

# print(len(res['ent']), len(res['x']), len(res['y']))
ent = torch.FloatTensor(res['ent'])
# print(type(res['x']))
x = torch.cat([w.unsqueeze(0) for w in res['x']], 0)
y = torch.LongTensor(res['y'])
# print(x.shape, y.shape)
_, indice = torch.sort(ent)
print(indice)

Updating Confidence indices...: 100%|██████████| 391/391 [00:13<00:00, 28.24it/s]


tensor([15820, 15937, 47699,  ...,  6957,  9987,  2892])


In [39]:
import numpy as np
np.save('/data/lijingru/DataFree/checkpoints/indice.npy', indice.numpy())